In [1]:
import pandas as pd
import torch
import optuna
from transformers import (
    RobertaTokenizer, 
    RobertaForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np



c:\Anaconda3\envs\daycon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 모델 및 토크나이저 로드
model_name = "DeepChem/ChemBERTa-77M-MLM"
tokenizer = RobertaTokenizer.from_pretrained(
    model_name,
    cache_dir='C:/huggingface_cache'
)
model = RobertaForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=1,
    cache_dir='C:/huggingface_cache'
)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

print(f"훈련 데이터: {train_df.shape}")
print(f"테스트 데이터: {test_df.shape}")



훈련 데이터: (1681, 3)
테스트 데이터: (100, 2)


In [4]:
# 데이터셋 클래스 정의
class SMILESDataset(Dataset):
    def __init__(self, smiles_list, labels=None):
        self.smiles_list = smiles_list
        self.labels = labels

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        smiles = str(self.smiles_list[idx])
        encoding = tokenizer(
            smiles,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        
        return item

# 평가 함수들
def normalized_rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse / (np.max(y_true) - np.min(y_true))

def pearson_correlation(y_true, y_pred):
    corr = np.corrcoef(y_true, y_pred)[0, 1]
    return np.clip(corr, 0, 1)

def competition_score(y_true, y_pred):
    nrmse = min(normalized_rmse(y_true, y_pred), 1)
    pearson = pearson_correlation(y_true, y_pred)
    return 0.5 * (1 - nrmse) + 0.5 * pearson

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()
    labels = labels.flatten()
    
    nrmse = normalized_rmse(labels, predictions)
    pearson = pearson_correlation(labels, predictions)
    comp_score = competition_score(labels, predictions)
    
    return {
        'nrmse': nrmse,
        'pearson': pearson,
        'competition_score': comp_score,
        'mse': mean_squared_error(labels, predictions)
    }



In [5]:
# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    train_df['Canonical_Smiles'].values,
    train_df['Inhibition'].values,
    test_size=0.2,
    random_state=42
)

# 데이터셋 생성
train_dataset = SMILESDataset(X_train, y_train)
val_dataset = SMILESDataset(X_val, y_val)
test_dataset = SMILESDataset(test_df['Canonical_Smiles'].values)

# 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



In [6]:
# Optuna 목적 함수
def objective(trial):
    # 하이퍼파라미터 탐색 공간
    learning_rate = trial.suggest_float("learning_rate", 5e-6, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    warmup_steps = trial.suggest_int("warmup_steps", 50, 300)
    weight_decay = trial.suggest_float("weight_decay", 0.001, 0.1, log=True)
    
    # 모델 로드
    model = RobertaForSequenceClassification.from_pretrained(
        model_name, 
        num_labels=1,
        cache_dir='C:/huggingface_cache'
    )
    
    # 훈련 설정
    training_args = TrainingArguments(
        output_dir=f'./optuna_trial_{trial.number}',
        num_train_epochs=30,  # 빠른 탐색을 위해 줄임
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        learning_rate=learning_rate,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="competition_score",
        greater_is_better=True,
        save_total_limit=1,
        report_to=None,
        fp16=False,
        dataloader_pin_memory=False
    )
    
    # Trainer 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # 빠른 탐색
    )
    
    # 훈련
    trainer.train()
    
    # 최종 평가
    eval_results = trainer.evaluate()
    
    # 메모리 정리
    del model
    torch.cuda.empty_cache()
    
    return eval_results['eval_competition_score']

In [7]:
# Optuna 스터디 생성 및 실행
print("Optuna 하이퍼파라미터 최적화 시작...")
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42)
)

# 최적화 실행 (50회 시도)
study.optimize(objective, n_trials=50, timeout=7200)  # 2시간 제한

print("최적화 완료!")
print(f"최고 점수: {study.best_value:.4f}")
print(f"최적 하이퍼파라미터: {study.best_params}")

[I 2025-07-06 11:45:52,806] A new study created in memory with name: no-name-5ed7e64e-5fbe-4246-9c0d-8e694b0063eb


Optuna 하이퍼파라미터 최적화 시작...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Nrmse,Pearson,Competition Score,Mse
1,1776.525000,1623.592896,0.405446,0.175801,0.385178,1623.593140
2,1630.925800,1439.815430,0.381810,0.193590,0.405890,1439.815430
3,1374.657000,1288.162842,0.361143,0.188377,0.413617,1288.162842
4,1322.749400,1171.611328,0.344418,0.201318,0.428450,1171.611328
5,1149.388100,1081.860718,0.330963,0.222468,0.445752,1081.860840
6,1087.598100,1010.719055,0.319896,0.234832,0.457468,1010.719055
7,1072.079300,954.806274,0.310922,0.242802,0.465940,954.806274
8,966.062500,908.967651,0.303367,0.246251,0.471442,908.967651
9,951.153700,872.809326,0.297272,0.260445,0.481586,872.809326
10,890.081100,843.409790,0.292222,0.319463,0.513620,843.409851


[I 2025-07-06 11:46:49,073] Trial 0 finished with value: 0.5136200510097674 and parameters: {'learning_rate': 2.8057582076672495e-05, 'batch_size': 8, 'warmup_steps': 89, 'weight_decay': 0.002051110418843397}. Best is trial 0 with value: 0.5136200510097674.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Nrmse,Pearson,Competition Score,Mse
1,1718.331200,1126.532104,0.337727,0.112826,0.387549,1126.532104
2,921.570500,744.817749,0.274612,0.156847,0.441118,744.817749
3,712.095300,698.374023,0.265912,0.254966,0.494527,698.374084
4,703.174100,680.636597,0.262514,0.221964,0.479725,680.636658
5,662.563400,639.080933,0.254374,0.319417,0.532522,639.080872
6,628.959500,626.038513,0.251765,0.329468,0.538852,626.038513
7,630.578200,621.255676,0.250801,0.377199,0.563199,621.255676
8,564.039700,614.229675,0.249379,0.387623,0.569122,614.229675
9,538.372700,631.563538,0.252873,0.394392,0.570760,631.563538
10,486.245000,634.914917,0.253543,0.390861,0.568659,634.914917


[I 2025-07-06 12:28:51,651] Trial 48 finished with value: 0.5816623837702477 and parameters: {'learning_rate': 0.00022069517200075023, 'batch_size': 8, 'warmup_steps': 168, 'weight_decay': 0.006166026366219402}. Best is trial 22 with value: 0.586500525139622.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Nrmse,Pearson,Competition Score,Mse
1,1701.007300,1051.018677,0.326212,0.146299,0.410044,1051.018677
2,881.043000,731.493896,0.272145,0.137796,0.432826,731.493896
3,708.513400,697.017090,0.265654,0.313500,0.523923,697.017090
4,702.075900,743.591125,0.274386,0.240924,0.483269,743.591125
5,679.701600,664.756470,0.259433,0.225745,0.483156,664.756470
6,650.912900,669.350281,0.260328,0.198273,0.468972,669.350220
7,654.595000,648.662537,0.256273,0.335316,0.539521,648.662537
8,583.616100,652.475891,0.257026,0.257835,0.500405,652.475830
9,596.672400,653.624573,0.257252,0.320581,0.531665,653.624512
10,529.323400,639.688171,0.254495,0.319885,0.532695,639.688171


[I 2025-07-06 12:29:37,092] Trial 49 finished with value: 0.5395211332783588 and parameters: {'learning_rate': 0.00023152889753023145, 'batch_size': 8, 'warmup_steps': 135, 'weight_decay': 0.0059508921263809215}. Best is trial 22 with value: 0.586500525139622.


최적화 완료!
최고 점수: 0.5865
최적 하이퍼파라미터: {'learning_rate': 0.00030994900647604996, 'batch_size': 8, 'warmup_steps': 201, 'weight_decay': 0.007364521772775286}


In [8]:
# 최적 하이퍼파라미터로 최종 훈련
print("\n최적 하이퍼파라미터로 최종 훈련 시작...")
best_params = study.best_params

training_args = TrainingArguments(
    output_dir='./chemberta_best',
    num_train_epochs=100,
    per_device_train_batch_size=best_params['batch_size'],
    per_device_eval_batch_size=best_params['batch_size'],
    warmup_steps=best_params['warmup_steps'],
    weight_decay=best_params['weight_decay'],
    learning_rate=best_params['learning_rate'],
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="competition_score",
    greater_is_better=True,
    save_total_limit=3,
    report_to=None,
    fp16=False,
    dataloader_pin_memory=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)]
)


최적 하이퍼파라미터로 최종 훈련 시작...


In [9]:
# 모델 훈련
print("모델 훈련 시작...")
trainer.train()

# 검증 성능 평가
print("검증 성능 평가...")
eval_results = trainer.evaluate()
print(f"최종 검증 성능: {eval_results}")



모델 훈련 시작...


Epoch,Training Loss,Validation Loss,Nrmse,Pearson,Competition Score,Mse
1,1517.462800,1074.737793,0.329872,0.189614,0.429871,1074.737793
2,706.320500,709.433289,0.268009,0.288829,0.510410,709.433289
3,718.493400,694.879333,0.265246,0.335674,0.535214,694.879333
4,712.618700,651.726074,0.256878,0.289535,0.516329,651.726074
5,626.076000,629.351624,0.252430,0.335201,0.541386,629.351624
6,644.044300,635.519409,0.253664,0.299992,0.523164,635.519409
7,579.373800,696.604309,0.265575,0.301205,0.517815,696.604309
8,589.841600,623.443420,0.251242,0.331755,0.540256,623.443420
9,565.954800,648.397461,0.256221,0.362771,0.553275,648.397461
10,539.277000,624.470947,0.251449,0.374752,0.561651,624.470886


검증 성능 평가...


최종 검증 성능: {'eval_loss': 620.6712646484375, 'eval_nrmse': 0.25068310833942326, 'eval_pearson': 0.3833638332789165, 'eval_competition_score': 0.5663403624697466, 'eval_mse': 620.6712646484375, 'eval_runtime': 0.354, 'eval_samples_per_second': 951.935, 'eval_steps_per_second': 121.463, 'epoch': 27.0}


In [10]:
# 전체 데이터로 재훈련
print("\n전체 데이터로 최종 훈련...")
full_dataset = SMILESDataset(train_df['Canonical_Smiles'].values, train_df['Inhibition'].values)

final_training_args = TrainingArguments(
    output_dir='./chemberta_final',
    num_train_epochs=25,
    per_device_train_batch_size=best_params['batch_size'],
    warmup_steps=best_params['warmup_steps'],
    weight_decay=best_params['weight_decay'],
    learning_rate=best_params['learning_rate'],
    logging_steps=50,
    save_strategy="no",
    report_to=None,
    fp16=False
)

final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=full_dataset,
    data_collator=data_collator,
)

final_trainer.train()


전체 데이터로 최종 훈련...


Step,Training Loss
50,430.359200
100,464.344800
150,450.707700
200,466.680700
250,477.232900
300,411.099100
350,432.048400
400,450.065000
450,398.023200
500,479.596800


TrainOutput(global_step=5275, training_loss=229.05776811500297, metrics={'train_runtime': 99.2923, 'train_samples_per_second': 423.245, 'train_steps_per_second': 53.126, 'total_flos': 45370363629480.0, 'train_loss': 229.05776811500297, 'epoch': 25.0})

In [11]:
print("테스트 데이터 예측 중...")
test_dataset = SMILESDataset(test_df['Canonical_Smiles'].values)
predictions = trainer.predict(test_dataset)
test_predictions = predictions.predictions.flatten()

# 결과 저장
submission['Inhibition'] = test_predictions
submission.to_csv('chemberta_optuna_submission.csv', index=False)

print(f"\n최적화 결과:")
print(f"- 최고 검증 점수: {study.best_value:.4f}")
print(f"- 최적 학습률: {best_params['learning_rate']:.2e}")
print(f"- 최적 배치 크기: {best_params['batch_size']}")
print(f"- 최적 Warmup Steps: {best_params['warmup_steps']}")
print(f"- 최적 Weight Decay: {best_params['weight_decay']:.4f}")
print("예측 결과 저장: chemberta_optuna_submission.csv")

테스트 데이터 예측 중...

최적화 결과:
- 최고 검증 점수: 0.5865
- 최적 학습률: 3.10e-04
- 최적 배치 크기: 8
- 최적 Warmup Steps: 201
- 최적 Weight Decay: 0.0074
예측 결과 저장: chemberta_optuna_submission.csv


In [12]:
# 예측값 분석
import pandas as pd
import numpy as np

# 결과 파일 로드
submission = pd.read_csv('chemberta_optuna_submission.csv')
train_df = pd.read_csv('train.csv')

print("=== 예측값 분석 ===")
print(f"예측값 범위: {submission['Inhibition'].min():.2f} ~ {submission['Inhibition'].max():.2f}")
print(f"실제값 범위: {train_df['Inhibition'].min():.2f} ~ {train_df['Inhibition'].max():.2f}")
print(f"예측값 평균: {submission['Inhibition'].mean():.2f}")
print(f"실제값 평균: {train_df['Inhibition'].mean():.2f}")
print(f"예측값 표준편차: {submission['Inhibition'].std():.2f}")
print(f"실제값 표준편차: {train_df['Inhibition'].std():.2f}")

print("\n=== 예측값 분포 ===")
print(submission['Inhibition'].describe())

print("\n=== 이상치 확인 ===")
print(f"음수값 개수: {(submission['Inhibition'] < 0).sum()}")
print(f"100 초과값 개수: {(submission['Inhibition'] > 100).sum()}")
print(f"NaN 개수: {submission['Inhibition'].isna().sum()}")

# 극단값 확인
print(f"\n가장 작은 5개 값: {submission['Inhibition'].nsmallest(5).values}")
print(f"가장 큰 5개 값: {submission['Inhibition'].nlargest(5).values}")

=== 예측값 분석 ===
예측값 범위: -0.60 ~ 74.70
실제값 범위: 0.00 ~ 99.38
예측값 평균: 29.62
실제값 평균: 33.22
예측값 표준편차: 20.12
실제값 표준편차: 26.41

=== 예측값 분포 ===
count    100.000000
mean      29.615916
std       20.120878
min       -0.595903
25%       13.647787
50%       24.885278
75%       44.081975
max       74.699570
Name: Inhibition, dtype: float64

=== 이상치 확인 ===
음수값 개수: 1
100 초과값 개수: 0
NaN 개수: 0

가장 작은 5개 값: [-0.5959028   0.13939762  1.3311311   1.8108578   1.9346439 ]
가장 큰 5개 값: [74.69957 74.68019 74.62001 73.23399 72.18682]
